# Create Dictionary of CIMIS Coordinates

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import ast
import csv
print('Done')

## CIMIS Data Load

In [ ]:
#Data Paths for CIMIS data WY19 All Stations
cimis_02_56 = '/content/drive/My Drive/Grad School/Stanford - Q3 2019-2020/CEE 260D/CEE260D Project/Data/CIMIS/CIMIS_Daily_WY19_id02-056.csv'
cimis_57_104 = '/content/drive/My Drive/Grad School/Stanford - Q3 2019-2020/CEE 260D/CEE260D Project/Data/CIMIS/CIMIS_Daily_WY19_id57-104.csv'
cimis_105_163 = '/content/drive/My Drive/Grad School/Stanford - Q3 2019-2020/CEE 260D/CEE260D Project/Data/CIMIS/CIMIS_Daily_WY19_id105-163.csv'
cimis_165_219 = '/content/drive/My Drive/Grad School/Stanford - Q3 2019-2020/CEE 260D/CEE260D Project/Data/CIMIS/CIMIS_Daily_WY19_id165-219.csv'
cimis_220_262 = '/content/drive/My Drive/Grad School/Stanford - Q3 2019-2020/CEE 260D/CEE260D Project/Data/CIMIS/CIMIS_Daily_WY19_id220-262.csv'

In [ ]:
cimis_1 = pd.read_csv(cimis_02_56)
cimis_2 = pd.read_csv(cimis_57_104)
cimis_3 = pd.read_csv(cimis_105_163)
cimis_4 = pd.read_csv(cimis_165_219)
cimis_5 = pd.read_csv(cimis_220_262)

## CIMIS API for Coordinates

In [ ]:
#CIMIS WEB API https://et.water.ca.gov/Rest/Index
cimis_api = 'https://et.water.ca.gov/api/station/'
api_key = '273cd116-fc49-462c-9104-ea513ccd1192'

In [ ]:
#Function for getting coordinates
def get_coordinates(df, dictionary):
  length = df.shape[0]

  for i in range(length):
    stn_name = df.iloc[i]['Stn Name']
    stn_nbr = df.iloc[i]['Stn Id']
    
    #add station to dictionary
    if stn_name not in dictionary:

      #find coordinates with api
      r = urllib.request.urlopen(cimis_api + str(stn_nbr))
      r = r.read()
      dict_str = r.decode('UTF-8')
      my_data = ast.literal_eval(dict_str)
      lat_str = my_data['Stations'][0]['HmsLatitude']
      lon_str = my_data['Stations'][0]['HmsLongitude']

      #convert lat/lon strings to int
      #lat
      slash1 = lat_str.find('/')
      lat_int = float(lat_str[slash1+1:])
      #lon
      slash2 = lon_str.find('/')
      lon_int = float(lon_str[slash2+1:])

      #add in values
      dictionary[stn_name] = [lat_int, lon_int]

In [ ]:
#create latitude and longitude dictionary
latlon_dict = {}

In [ ]:
get_coordinates(cimis_1, latlon_dict)
get_coordinates(cimis_2, latlon_dict)
get_coordinates(cimis_3, latlon_dict)
get_coordinates(cimis_4, latlon_dict)
get_coordinates(cimis_5, latlon_dict)

In [ ]:
latlon_dict

In [ ]:
a_file = open('latlon_dict.csv', 'w')

writer = csv.writer(a_file)
for key, value in latlon_dict.items():
  writer.writerow([key, value])

a_file.close()

In [ ]:
#Uncomment these cells to download dictionary csv file

#from google.colab import files
#files.download('latlon_dict.csv')

## Cimis FivePoints TimeSeries Example

In [ ]:
fivepoints = cimis_1.loc[cimis_1['Stn Name'] == 'FivePoints']
fivepoints.columns

In [ ]:
x = fivepoints['Date']
y = fivepoints['ETo (mm)']

plt.figure(figsize=(10,10))
plt.plot(x,y)
plt.xlabel('Days')
plt.ylabel('ETo (mm)')
plt.title('FivePoints')